# Linear Regression + Pipeline on PYNQ-Z1
### n_features = 32, n_outputs=10

In [1]:
import numpy as np
from sklearn import datasets
from pynq_sklearn.linear_model import PynqLinearRegression
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn.kernel_approximation import RBFSampler
import time


# Generate dataset of Ints
X, y = datasets.make_regression(n_samples=5000, n_features=10, n_targets=10, random_state=0, noise=4.0,
                       bias=100.0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, random_state=42)
X_test = np.abs((X_test*(2**20)).astype("int32"))
y_test = np.abs((y_test*(2**20)).astype("int32"))


#### 1. Define stages of Pipeline

In [2]:
lin = PynqLinearRegression(fit_intercept=True, hw_accel=True) # offloaded to FPGA
feature_map_fourier = RBFSampler(gamma=.2, random_state=1) # Kept in SW for now

#### 2. Create Pipeline and set_params

In [3]:
fourier_approx_pipe = pipeline.Pipeline([("feature_map", feature_map_fourier), ("reg", lin)])
fourier_approx_pipe.set_params(feature_map__n_components=32)

Pipeline(memory=None,
     steps=[('feature_map', RBFSampler(gamma=0.2, n_components=32, random_state=1)), ('reg', PynqLinearRegression(hw_accel=True))])

#### 3. Fit()

In [4]:
fourier_approx_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('feature_map', RBFSampler(gamma=0.2, n_components=32, random_state=1)), ('reg', PynqLinearRegression(hw_accel=True))])

#### 4. Predict and score

In [5]:
fourier_score = fourier_approx_pipe.score(X_test, y_test)
print("score: ", fourier_score)

Offloading predict to FPGA ...
score:  0.9999999982674712
